In [1]:
import tensorflow as tf
tf.__version__

'1.13.1'

*after we import TensorFlow (with import tensorflow as tf ), a specific*
*empty default graph is formed. All the nodes we create are automatically associated*
*with that default graph.*

In [2]:
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

In [3]:
d = tf.multiply(a, b)
e = tf.add(c, b)
f = tf.subtract(d, e)

In [4]:
sess = tf.Session()
outs = sess.run(f)
sess.close()
print("outs = {}".format(outs))

outs = 5


In [5]:
print(tf.get_default_graph())
g = tf.Graph()
print(g)

##### The with statement
* *The with statement is used to wrap the execution of a block with
methods defined by a context manager—an object that has the spe‐
cial method functions .__enter__() to set up a block of code
and .__exit__() to exit the block.*
* *In layman’s terms, it’s very convenient in many cases to execute
some code that requires “setting up” of some kind (like opening a
file, SQL table, etc.) and then always “tearing it down” at the end,
regardless of whether the code ran well or raised any kind of excep‐
tion. In our case we use with to set up a graph and make sure*

##### Fetches
* *In our initial graph example, we request one specific node (node f ) by passing the
variable it was assigned to as an argument to the sess.run() method. This argument
is called fetches , corresponding to the elements of the graph we wish to com‐
pute.*

In [6]:
with tf.Session() as sess:
    fetches = [a, b, c, d, e, f]
    outs = sess.run(fetches)
print("outs = {}".format(outs))
print(type(outs[0]))

outs = [5, 2, 3, 10, 5, 5]
<class 'numpy.int32'>


### Flowing Tensors
##### Nodes are operations, Edges are tensor objects

##### A note on constuctors
*The tf.operator function could be thought of as a constructor,
but to be more precise, this is actually not a constructor at all, but
rather a factory method that sometimes does quite a bit more than
just creating the operator objects.*

In [7]:
import numpy as np
c = tf.constant([[1,2,3],
                [4,5,6]])
print("Python List input: {}".format(c.get_shape()))
c = tf.constant(np.array([
        [[1,2,3],
        [4,5,6]],
    
        [[1,1,1],
        [2,2,2]]
]))
print("3d NumPy array input: {}".format(c.get_shape()))

Python List input: (2, 3)
3d NumPy array input: (2, 2, 3)


In [8]:
sess = tf.InteractiveSession()
c = tf.lin_space(0.0, 4.0, 5)
print("The content of 'c':\n{}\n".format(c.eval()))
sess.close()

The content of 'c':
[0. 1. 2. 3. 4.]



##### Interactive sessions
*tf.InteractiveSession() allows you to replace the usual tf.Ses
sion() , so that you don’t need a variable holding the session for
running ops. This can be useful in interactive Python environ‐
ments, like when writing IPython notebooks, for instance.*

In [9]:
A = tf.constant([ [1,2,3],
                  [4,5,6] ])
print(A.get_shape())

x = tf.constant([1,0,1])
print(x.get_shape())

(2, 3)
(3,)


In [10]:
x = tf.expand_dims(x, 1)
print(x.get_shape())

b = tf.matmul(A, x)

sess = tf.InteractiveSession()
print('matmul result:\n {}'.format(b.eval()))
sess.close()

(3, 1)
matmul result:
 [[ 4]
 [10]]


In [11]:
init_val = tf.random_normal((1, 5), 0, 1)
var = tf.Variable(init_val, name='var')
print("pre run: \n{}".format(var))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    post_var = sess.run(var)
    
print("\npost run: \n{}".format(post_var))

Instructions for updating:
Colocations handled automatically by placer.
pre run: 
<tf.Variable 'var:0' shape=(1, 5) dtype=float32_ref>

post run: 
[[-1.3886242   1.903709   -0.17332391 -0.5504384  -0.81537664]]


In [12]:
x_data = np.random.randn(5, 10)
w_data = np.random.randn(10, 1)
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape=(5, 10))
    w = tf.placeholder(tf.float32, shape=(10, 1))
    b = tf.fill((5, 1), -1.)
    xw = tf.matmul(x, w)
    
    xwb = xw + b
    s = tf.reduce_max(xwb)
    with tf.Session() as sess:
        outs = sess.run(s, feed_dict={x: x_data, w:w_data})

print("outs = {}".format(outs))

outs = -1.4228520393371582


##### linear regression model

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y_true = tf.placeholder(tf.float32, shape=None)
w = tf.Variable([[0, 0, 0]], dtype=tf.float32, name='weights')
b = tf.Variable(0, dtype=tf.float32, name='bias')

In [14]:
y_pred = tf.matmul(w, tf.transpose(x)) + b

##### MSE and cross entropy

In [15]:
loss = tf.reduce_mean(tf.square(y_true - y_pred))

In [16]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
loss = tf.reduce_mean(loss)

##### The gradient descent optimizer

In [17]:
learning_rate = 0.3
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use tf.cast instead.


### Example 1: Linear regression

In [18]:
x_data = np.random.randn(2000, 3)
w_real = [0.3, 0.5, 0.1]
b_real = -0.2

noise = np.random.randn(1, 2000) * 0.5
y_data = np.matmul(w_real, x_data.T) + b_real + noise

In [19]:
NUM_STEPS = 10
g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3])
    y_true = tf.placeholder(tf.float32, shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0, 0, 0]], dtype=tf.float32, name='weights')
        b = tf.Variable(0, dtype=tf.float32, name='bias')
        y_pred = tf.matmul(w, tf.transpose(x)) + b
        
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true - y_pred))
    
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)
    
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for step in range(NUM_STEPS):
            sess.run(train, {x: x_data, y_true: y_data})
            print(step, sess.run([w, b]))
            wb_.append(sess.run([w, b]))
        print(10, sess.run([w, b]))

0 [array([[0.28154302, 0.512231  , 0.08864806]], dtype=float32), -0.20016277]
1 [array([[0.2965384 , 0.5129814 , 0.11048459]], dtype=float32), -0.20515633]
2 [array([[0.2975108 , 0.51346946, 0.11114071]], dtype=float32), -0.20588213]
3 [array([[0.29756877, 0.51348466, 0.11119376]], dtype=float32), -0.2059025]
4 [array([[0.29757252, 0.5134857 , 0.11119597]], dtype=float32), -0.20590414]
5 [array([[0.29757276, 0.5134858 , 0.11119611]], dtype=float32), -0.20590422]
6 [array([[0.2975728 , 0.5134858 , 0.11119611]], dtype=float32), -0.2059042]
7 [array([[0.2975728 , 0.5134858 , 0.11119611]], dtype=float32), -0.2059042]
8 [array([[0.2975728 , 0.5134858 , 0.11119611]], dtype=float32), -0.2059042]
9 [array([[0.2975728 , 0.5134858 , 0.11119611]], dtype=float32), -0.2059042]
10 [array([[0.2975728 , 0.5134858 , 0.11119611]], dtype=float32), -0.2059042]


### Example 2: Logistic regression

In [20]:
N = 20000

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

x_data = np.random.randn(N, 3)
w_real = [0.3, 0.5, 1]
b_real = -0.2
wxb = np.matmul(w_real, x_data.T) + b_real

y_data_pre_noise = sigmoid(wxb)
y_data = np.random.binomial(1, y_data_pre_noise)

In [21]:
NUM_STEPS = 50
with tf.name_scope('loss') as scope:
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred)
    loss = tf.reduce_mean(loss)
# Before starting, initialize the variables. We will 'run' this first.
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for step in range(NUM_STEPS):
        sess.run(train,{x: x_data, y_true: y_data})
        if (step % 5 == 0):
            print(step, sess.run([w,b]))
            wb_.append(sess.run([w,b]))
    print(50, sess.run([w,b]))

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 3), dtype=float32) is not an element of this graph.